In [23]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [24]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import model_selection, metrics   #Additional scklearn functions

In [25]:
data = pd.read_csv('./training_data_v2.csv', low_memory=False)

In [26]:
data.head()

,application_key,mvar1,mvar2,mvar3,mvar4,mvar5,mvar6,mvar7,mvar8,mvar9,...,mvar36,mvar37,mvar38,mvar39,mvar42,mvar43,mvar44,mvar46,mvar47,default_ind
0,230032,1696,1.6541,0.000,0.0,0.0,0,6015,322,40369,...,8,10,4,1,0.08696,10,0.63899,0,1,0
1,230033,1846,0.8095,0.000,0.0,0.0,102,7532,3171,18234,...,8,0,2,0,0.00000,13,0.63836,0,0,1
2,230035,1739,0.2193,0.000,0.0,0.0,1982,26440,4955,20316,...,3,3,2,0,0.15385,3,0.53241,0,0,0
3,230036,1787,0.0118,0.225,0.0,0.0,5451,5494,5494,7987,...,2,3,2,0,0.00000,1,0.92665,0,0,0
4,230040,1836,0.1358,0.000,0.0,0.0,347,38964,17828,70729,...,6,12,2,0,0.00000,10,0.89868,0,0,0


In [27]:
X = data[data.loc[:, data.columns != 'default_ind'].columns]
Y = data['default_ind']
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X, Y, test_size=.2, random_state=1)

In [28]:
X_train['default_ind'] = Y_train

/home/epsilon/.virtualenvs/DATA/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [29]:
X_train.head()
target = 'default_ind'

In [30]:
np.sum(X_train['default_ind'].values)

10074

In [31]:
np.sum(Y_train.values)

10074

In [32]:
def modelfit(alg, dtrain, predictors,useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        #xgb_param['num_class']=2
        xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtrain[target].values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
             early_stopping_rounds=early_stopping_rounds,stratified=True)
        alg.set_params(n_estimators=cvresult.shape[0])
    
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain[target],eval_metric='auc')
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]
        
    #Print model report:
    print ("\nModel Report")
    print ("Accuracy : %.4g" % metrics.accuracy_score(dtrain[target].values, dtrain_predictions))

In [33]:
predictors = [x for x in X_train.columns if x not in ['default_ind']]
print(predictors)

['application_key', 'mvar1', 'mvar2', 'mvar3', 'mvar4', 'mvar5', 'mvar6', 'mvar7', 'mvar8', 'mvar9', 'mvar10', 'mvar12', 'mvar13', 'mvar14', 'mvar16', 'mvar17', 'mvar18', 'mvar19', 'mvar20', 'mvar21', 'mvar24', 'mvar25', 'mvar26', 'mvar27', 'mvar28', 'mvar29', 'mvar32', 'mvar33', 'mvar34', 'mvar36', 'mvar37', 'mvar38', 'mvar39', 'mvar42', 'mvar43', 'mvar44', 'mvar46', 'mvar47']


In [34]:
xgb1 = XGBClassifier(
 learning_rate =0.01,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)
modelfit(xgb1, X_train, predictors)


Model Report
Accuracy : 0.8377


In [36]:
Y_pred=xgb1.predict(X_test)

In [37]:
xgb1.predict_proba(X_test)

array([[0.93748975, 0.06251027],
       [0.89933276, 0.10066727],
       [0.6918422 , 0.30815783],
       ...,
       [0.9848882 , 0.0151118 ],
       [0.97661865, 0.02338136],
       [0.9011196 , 0.09888041]], dtype=float32)

In [38]:
float(np.sum(Y_pred == Y_test))/float(np.sum(Y_test == Y_test))

0.8303929430633521

In [39]:
metrics.accuracy_score(Y_test, Y_pred)

0.8303929430633521

In [40]:
X_lead = pd.read_csv('refined_leaderboard_data.csv', low_memory=False)

In [41]:
X_lead.head()

,application_key,mvar1,mvar2,mvar3,mvar4,mvar5,mvar6,mvar7,mvar8,mvar9,...,mvar34,mvar36,mvar37,mvar38,mvar39,mvar42,mvar43,mvar44,mvar46,mvar47
0,350053.0,1753.0,0.5001,0.000,0.0,0.0,206.000000,19179.000000,206.000000,14221.000000,...,0.0,3.0,11.0,1.0,0.000000,0.42105,3.0,0.856610,0.000000,0.0
1,350054.0,1894.0,1.9701,0.000,0.0,0.0,297.000000,19820.000000,19820.000000,176895.000000,...,0.0,3.0,12.0,3.0,0.000000,0.00000,15.0,0.943910,0.000000,1.0
2,350055.0,1774.0,0.1718,0.000,0.0,0.0,197.000000,2563.000000,877.000000,8869.000000,...,0.0,3.0,4.0,1.0,0.000000,0.33333,3.0,0.764670,0.000000,1.0
3,350056.0,1586.0,0.1123,5.299,0.0,0.0,1624.069192,17750.368864,6753.202313,33825.554662,...,0.0,1.0,0.0,1.0,0.089912,1.50000,0.0,0.702558,0.092712,1.0
4,350057.0,1832.0,1.4442,0.000,0.0,0.0,179.000000,7577.000000,179.000000,21059.000000,...,0.0,17.0,0.0,3.0,0.000000,0.00000,23.0,0.791900,0.000000,0.0


In [42]:
Y_lead=pd.DataFrame(xgb1.predict(X_lead))

In [43]:
sol = pd.DataFrame()
sol['application_key'] = X_lead['application_key'].astype(dtype='int')
sol['default_ind'] = Y_lead.astype(dtype='int')

In [110]:
sol.to_csv(path_or_buf='./Submissions/Analyzing_IITGuwahati_22.csv', index = False, header=False)